In [2]:
import os
import pandas as pd
# requires dammit env
# source activate dammit
from dammit.fileio.gff3 import GFF3Parser

# Make list of genes being quantified

Salmon was run with the de novo transcriptome assemblies, with the original Trinity ID in the header for each fasta contig record, like this (example salmon output file):

```
Name	Length	EffectiveLength	TPM	NumReads
TRINITY_DN2202_c0_g1_i1	210	39.818	2.683835	2.000000
TRINITY_DN2270_c0_g1_i1	213	41.064	0.000000	0.000000
TRINITY_DN2201_c0_g1_i1	266	69.681	0.766816	1.000000
TRINITY_DN2222_c0_g1_i1	243	55.794	2.873014	3.000000
TRINITY_DN2291_c0_g1_i1	245	56.916	0.000000	0.000000
TRINITY_DN2269_c0_g1_i1	294	89.251	0.000000	0.000000
TRINITY_DN2269_c1_g1_i1	246	57.479	0.000000	0.000000
TRINITY_DN2279_c0_g1_i1	426	207.443	0.000000	0.000000
TRINITY_DN2262_c0_g1_i1	500	280.803	0.190459	1.000912
TRINITY_DN2253_c0_g1_i1	1523	1303.116	0.164015	4.000000
TRINITY_DN2287_c0_g1_i1	467	247.962	0.000000	0.000000
TRINITY_DN2287_c1_g1_i1	325	113.826	0.469425	1.000000
TRINITY_DN2237_c0_g1_i1	306	98.441	0.542788	1.000000
TRINITY_DN2237_c0_g2_i1	307	99.229	0.000000	0.000000
TRINITY_DN2250_c0_g1_i1	368	151.832	0.000000	0.000000
TRINITY_DN2250_c1_g1_i1	271	72.988	0.000000	0.000000
TRINITY_DN2208_c0_g1_i1	379	162.080	1.978014	6.000000
TRINITY_DN2277_c0_g1_i1	269	71.657	0.745677	1.000000
TRINITY_DN2231_c0_g1_i1	209	39.409	0.000000	0.000000
TRINITY_DN2231_c1_g1_i1	334	121.411	0.000000	0.000000
TRINITY_DN2204_c0_g1_i1	287	84.121	0.000000	0.000000
```

Takes each salmon output .quant file (like the one above), parse the "i" (isoform) off of the ID to get a list of genes and isoforms identified by Trinity.

In [4]:
species_dirs = "../salmon_denovo_by_species"
out_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/"
dirs = os.listdir(species_dirs)
for species in dirs:
    if species != ".DS_Store":
        species_gene_file = species + "_gene_transcript_table.txt"
        files = os.listdir(species_dirs + "/" + species)
        for quant_dir in files:
            if quant_dir != ".DS_Store":
                replicate = quant_dir.split(".")[0]
                print(replicate)
                quant_files = os.listdir(species_dirs + "/" + species + "/" + quant_dir)
                for file in quant_files:
                    if file.endswith(".sf"):
                        with open(species_dirs + "/" + species + "/" + quant_dir + "/" + file) as qf:
                            header = next(qf).split("\t")
                            expression_quant_data = qf.readlines()
                        with open(out_dir + species_gene_file,"w") as gt:
                            for line in expression_quant_data:
                                transcript = line.split("\t")[0]
                                gene = transcript[:-3]
                                gt.write(transcript + "\t")
                                gt.write(gene + "\n")

L_goodei_transfer_2
L_goodei_BW_3
L_goodei_BW_1
L_goodei_transfer_3
L_goodei_transfer_1
L_goodei_BW_2
L_goodei_FW_2
L_goodei_FW_1
L_goodei_FW_3
F_notti_FW_2
F_notti_FW_1
L_parva_FW_3
L_parva_FW_1
L_parva_transfer_3
L_parva_transfer_1
L_parva_FW_2
L_parva_transfer_2
L_parva_BW_2
L_parva_BW_1
L_parva_BW_3
F_similis_transfer_1
F_similis_BW_1
F_similis_transfer_3
F_similis_BW_3
F_similis_BW_2
F_similis_transfer_2
F_similis_FW_2
F_similis_FW_3
F_similis_FW_1
F_olivaceous_FW_2
F_olivaceous_transfer_2
F_olivaceous_FW_1
F_olivaceous_transfer_1
F_olivaceous_FW_3
F_olivaceous_BW_3
F_olivaceous_BW_1
F_olivaceous_BW_2
F_notatus_FW_2
F_notatus_FW_3
F_notatus_FW_1
F_notatus_transfer_1
F_notatus_transfer_3
F_notatus_BW_1
F_notatus_BW_3
F_notatus_transfer_2
F_notatus_BW_2
F_heteroclitusMDPP_FW_3
F_heteroclitusMDPP_FW_1
F_heteroclitusMDPP_FW_2
F_heteroclitusMDPP_BW_2
F_heteroclitusMDPP_transfer_2
F_heteroclitusMDPP_transfer_3
F_heteroclitusMDPP_BW_1
F_heteroclitusMDPP_transfer_1
F_heteroclitusMDPP_BW_3

# Uses the dammit's `GFF3Parser` function 
1. Digests the gff3 file for each species (output from dammit, downloaded from the farm cluster)
2. Sorts each contig by E-value score
3. Assigns the lowest E-value score for each contig 
4. Separately, for each contig, saves gene names from the NCBI annotated F. heteroclitus genome

Writes 2 separate .csv files:

* (species)_Fhet_genenames.csv
* (species)_onegenenamepertranscript.csv

In [5]:
annotations_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/gff_annotations/"
annotations = os.listdir(annotations_dir)
print(annotations)
for annotation in annotations:
    if annotation != ".DS_Store":
        species = annotation.split(".")[0]
        print(annotation)
        name = annotations_dir + annotation
        annotations = GFF3Parser(filename=name).read()
        annotations = annotations.dropna(subset=['Name'])
        pickonename = annotations.sort_values(by=['seqid', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name']]
        pickonename = pickonename.dropna(axis=0,how="all")
        fund = annotations[annotations['Name'].str.startswith("gi")]
        names = fund.sort_values(by=['seqid'], ascending=True)[['seqid', 'Name']]
        fund_names = "/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/" + species + "_Fhet_genenames.csv"
        pickone = "/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/" + species + "_onegenenamepertranscript.csv"
        names.to_csv(fund_names)
        pickonename.to_csv(pickone)
        

['F_notatus.trinity_out.Trinity.fasta.dammit.gff3', '.DS_Store', 'F_rathbuni.trinity_out.Trinity.fasta.dammit.gff3', 'F_diaphanus.trinity_out.Trinity.fasta.dammit.gff3', 'F_chrysotus.trinity_out.Trinity.fasta.dammit.gff3', 'L_goodei.trinity_out.Trinity.fasta.dammit.gff3', 'F_heteroclitusMDPL.trinity_out.Trinity.fasta.dammit.gff3', 'F_heteroclitusMDPP.trinity_out.Trinity.fasta.dammit.gff3', 'F_similis.trinity_out.Trinity.fasta.dammit.gff3', 'L_parva.trinity_out.Trinity.fasta.dammit.gff3', 'A_xenica.trinity_out.Trinity.fasta.dammit.gff3', 'F_catanatus.trinity_out.Trinity.fasta.dammit.gff3', 'F_zebrinus.trinity_out.Trinity.fasta.dammit.gff3', 'F_parvapinis.trinity_out.Trinity.fasta.dammit.gff3', 'F_grandis.trinity_out.Trinity.fasta.dammit.gff3', 'F_sciadicus.trinity_out.Trinity.fasta.dammit.gff3', 'F_olivaceous.trinity_out.Trinity.fasta.dammit.gff3', 'F_notti.trinity_out.Trinity.fasta.dammit.gff3']
F_notatus.trinity_out.Trinity.fasta.dammit.gff3


/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):
/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


F_rathbuni.trinity_out.Trinity.fasta.dammit.gff3
F_diaphanus.trinity_out.Trinity.fasta.dammit.gff3
F_chrysotus.trinity_out.Trinity.fasta.dammit.gff3
L_goodei.trinity_out.Trinity.fasta.dammit.gff3
F_heteroclitusMDPL.trinity_out.Trinity.fasta.dammit.gff3
F_heteroclitusMDPP.trinity_out.Trinity.fasta.dammit.gff3
F_similis.trinity_out.Trinity.fasta.dammit.gff3
L_parva.trinity_out.Trinity.fasta.dammit.gff3
A_xenica.trinity_out.Trinity.fasta.dammit.gff3
F_catanatus.trinity_out.Trinity.fasta.dammit.gff3
F_zebrinus.trinity_out.Trinity.fasta.dammit.gff3
F_parvapinis.trinity_out.Trinity.fasta.dammit.gff3
F_grandis.trinity_out.Trinity.fasta.dammit.gff3
F_sciadicus.trinity_out.Trinity.fasta.dammit.gff3
F_olivaceous.trinity_out.Trinity.fasta.dammit.gff3
F_notti.trinity_out.Trinity.fasta.dammit.gff3


# Combines multiple annotation types into a signle file with multiple columns per contig

Requires the dammit output `Trinity.fasta.dammit.namemap.csv` file from each species (downloaded from the farm cluster). This file connects the Trinity ID (e.g. `TRINITY_DN2202_c0_g1_i1`) to the dammit id (e.g. `Transcript_286223`).

Connects each annotation type (lowest e-value, Fundulus heteroclitus gene name, Trinity gene, Trinity contig, dammit id) and combines them into one single file. This is then used as input to the differential expression analysis to connect the differentially expressed genes.

The output below displays the shape of the 4 dataframes:

* dammit namemap + parsed Trinity gene and isoform ID (this is equal to the number of contigs in the assembly)
* number of Fundulus heteroclitus names matching (multiple per contig, e.g. isoforms)
* after merging the dammit namemap with the one gene name per contig result from above (outer = union) (this number tends to be the same as the first number above
* after merging the merged frame with the Fundulus heteroclitus gene names (union of 2nd and 3rd dataframes above)

In [27]:
namemap_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/"
maps = os.listdir(namemap_dir)
for namemap in maps:
    if namemap.endswith("dammit.namemap.csv"):
        species = namemap.split(".")[0]
        namemap2 = namemap_dir + species + "_namemap2.csv"
        with open(namemap_dir + namemap) as f:
            header = next(f)
            new_header = header.strip() + ","+"seqid"+","+"Name"
            ids = f.readlines()
        with open(namemap2,"w") as n:
            n.write(new_header+"\n")
            for line in ids:
                trinity_full = line.split(",")[0]
                renamed = line.split(",")[-1].strip()
                trinity_contig = trinity_full.split(" ")[0].strip('"')
                gene = trinity_contig[:-3]
                n.write(trinity_contig+",")
                n.write(renamed+",")
                n.write(gene+",")
                n.write(gene+"\n")
        annotations = namemap_dir + species + "_onegenenamepertranscript.csv"
        Fhet = namemap_dir + species + "_Fhet_genenames.csv"
        ann = pd.read_csv(annotations)
        dammit_Trinity = pd.read_csv(namemap2)
        print(namemap2)
        print(dammit_Trinity.shape)
        species_Fhet = pd.read_csv(Fhet)
        print(Fhet)
        print(species_Fhet.shape)
        species_Fhet = species_Fhet.drop(['Unnamed: 0'], axis=1)
        ann = ann.drop(['Unnamed: 0'], axis=1)
        combined = pd.merge(dammit_Trinity, ann, how='outer', left_on="renamed", right_on="seqid")
        print(combined.shape)
        combined = combined.drop(['renamed'],axis=1)
        combined = pd.merge(combined,species_Fhet,how = "outer",left_on="seqid_y",right_on="seqid")
        print(combined.shape)
        combined['Name_y'] = combined['Name_y'].fillna(combined['seqid_x'])
        combined = combined.drop(['Name_x'],axis=1)
        new = combined.to_csv("/Users/johnsolk/Documents/UCDavis/Whitehead/annotation_gene_names/" + species + "_gene_names.csv")
        #print("/Users/johnsolk/Documents/UCDavis/Whitehead/annotation_gene_names/" + species + "_gene_names.csv")

/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_diaphanus_namemap2.csv
(384218, 4)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_diaphanus_Fhet_genenames.csv
(134129, 3)
(384218, 6)
(455093, 7)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_chrysotus_namemap2.csv
(396400, 4)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_chrysotus_Fhet_genenames.csv
(168861, 3)
(396400, 6)
(485204, 7)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/L_goodei_namemap2.csv
(385476, 4)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/L_goodei_Fhet_genenames.csv
(168019, 3)
(385476, 6)
(478428, 7)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_heteroclitusMDPL_namemap2.csv
(592419, 4)
/Users/johnsolk/Documents/UCDavis/Whitehead/intermediate_denovo_annotation_files/F_